<a href="https://colab.research.google.com/github/tranvietphi20104051/baitapAI/blob/main/phanmain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import cv2
from modules.text_recognition.vietocr.tool.predictor import Predictor
from modules.text_recognition.vietocr.tool.config import Cfg
from modules.detect_word import OCR
from modules.crop_image import CropImg


def vietnamese_ocr():
    config = Cfg.load_config_from_name('vgg_transformer')
    config['weights'] = './models/transformerocr.pth'
    config['cnn']['pretrained'] = False
    config['device'] = 'cpu'
    config['predictor']['beamsearch'] = False
    detector = Predictor(config)
    return detector


def id_card_ocr(path, run=False):
    if run:
        IMG_SIZE = 640
        model_crop = torch.hub.load('ultralytics/yolov5', 'custom', path='models/yolov5_l6_cccd.pt')
        detect_model = torch.hub.load('ultralytics/yolov5', 'custom', path='models/information_yolov5l.pt')
        nlp_model = vietnamese_ocr()
        img = cv2.imread(path)
        crop = model_crop(img, IMG_SIZE)

        for i in range(len(crop.xyxy)):
            for i in range(len(crop.xyxy)):
                image_crop = CropImg(crop.xyxy[i], crop.pandas().xyxy[i], img)
                detect_id_card = detect_model(image_crop, IMG_SIZE)
                print(detect_id_card)
                for j in range(len(detect_id_card.xyxy)):
                    dictionary = OCR(detect_id_card.xyxy[j], detect_id_card.pandas().xyxy[j], image_crop, nlp_model)
                    return dictionary


if __name__ == '__main__':
    id_card_ocr(run=False)
